## Adaptive Grasping

This skill may be the most sensitive as the gripper driver is not really stable against using the tool-changer usage.

In (programming) theory, the gripper-driver is started via 

```bash
roslaunch hrr_cobot_robot hrr_wsg_50.launch
```

via the URDF-updater, i.e. every time the tool is set to 

```bash
rosparam set /hrr_cobot/tool_name "wsg_50_dsa"
```

or 

```python
cobot.change_tool("wsg_50_dsa")
```

but reality is different, so the rqt-graph below shows the skill-server with all nodes for the sensitive grasping

![rqt-graph](./media/hrr_wsg_setup_rqt.png)

**NOTE: The velocity based variant requires ```do-mpc```, so you need to run 
```pip install do-mpc``` to replicate the setup below**



### Robot setup without skill-server

![rqt-graph](./media/rqt_without_skill_server.png)


In [1]:
# copied from skill-server notebook -> can be ignored
import pathlib
import numpy as np
import spatialmath as sm

# ROS
import actionlib
import rospy
import hrr_common

from hr_recycler_msgs.msg import AdaptiveGraspingAction, AdaptiveGraspingGoal

np.set_printoptions(precision=5, suppress=True)

hrr_common.utils.set_ros_environment(ros_master='hrrcobotLinux54')
rospy.init_node('grasping_client')

cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")

def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client

def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()

current hostname:	hrrcobotLinux54
current IP:      	129.187.147.188
ROS-MASTER-URI:  	http://hrrcobotLinux54:11311


In [4]:
## pycharm fucked up
import copy
import hrr_cobot_robot
from hr_recycler_msgs.msg import ToolType
from hrr_cobot_robot.manipulation_skills.lightbulb_grasping import SimpleMp
cobot = hrr_cobot_robot.HrrCobotControl.from_ros(cobot_prefix="/hrr_cobot", compile_numba=False)
cobot.change_tool("wsg_50_dsa")

pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


True

In [5]:
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[-14.62622  -3.30371  28.27515] [N]
current torque:	[ -0.9173  -10.54138   0.55615] [Nm]
current wrench:	[-14.60883  -3.30719  28.31562  -0.93104 -10.56159   0.54858] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[ -2.97543   5.161   -91.02456   1.88446  81.43126  -0.37623][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.47519 0.02209 0.66174][m]
quat:		0.02126 <[-0.02478  0.99934 -0.01569]>
tool-pos:	[0.47519 0.02209 0.66174][m]
tool-rpy:	[177.1964    2.47995 178.26144][°]
robot-status:	ready
---
tool set to wsg_50_dsa
E_p_EC:=[0.     0.     0.2445]
R_E_C:=[[ 0.70711 -0.70711  0.     ]
 [ 0.70711  0.70711  0.     ]
 [ 0.       0.       1.     ]]
---


In [7]:
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)

In [8]:
grasp_goal = AdaptiveGraspingGoal()
T_B_C_test = sm.SE3(0.66, 0.0, 0.1)
grasp_goal.object_center.pose.position = hrr_common.np2vec3(T_B_C_test.t)
grasp_goal.object_center.pose.orientation = hrr_common.np2quat(hrr_common.rotmat2quat(T_B_C_test.R))
grasp_goal.contact_force = 5.0
grasp_goal.timeout = 2000.0

In [13]:
grasping_client.send_goal(grasp_goal)

[rosout] Received new goal for sensitive_grasping-skill.


[rosout] Sensitive Grasping -> received none-positive goal-argument gripper_open_width:=0.0. Use default 0.06
[rosout] Sensitive Grasping -> received none-positive goal-argument tilting_angle:=0.0. Use default 1.5707963267948966
[rosout] no solution found for pose:
   0.7071    0.7071    0         0.66      
  -0.7071    0.7071    0         0         
   0         0         1        -0.1445    
   0         0         0         1         

[rosout] final destination pose    1         0         0         0.66      
   0         1         0         0         
   0         0         1         0.1       
   0         0         0         1         
 is not reachable
[rosout] Got a result when we were already in the DONE state


In [12]:
cancel(grasping_client)

In [14]:
cobot.B_F_msr

array([ 10.59521, -14.46739, -26.62064,  -7.21665,  -9.41109,  -0.53413])

## development

pycharm seemed buggy on the robot PC, so continue here

In [6]:
grasping = hrr_cobot_robot.SensitiveGrasping.from_ros(cobot=cobot, cobot_prefix="/hrr_cobot",
                                          skill_prefix="/hrr_cobot/sensitive_grasping")
grasping.pre_skill_execution(ToolType.WSG_50_DSA, hardcoded_transformation=False)

[rosout] [/grasping_client] could not get ROS-parameter /grasping_client/buffer_size. Use default 30000 instead.
[rosout] [/grasping_client] could not get ROS-parameter /grasping_client/observe_attributes. Use default ['q', 'B_F_msr', 'B_err_F'] instead.
[rosout] [/grasping_client] could not get ROS-parameter /hrr_cobot/sensitive_grasping/drop_off_hover_distance. Use default 0.15 instead.
[rosout] [/grasping_client] could not get ROS-parameter /hrr_cobot/sensitive_grasping/f_lift. Use default 15.0 instead.
[rosout] [/grasping_client] could not get ROS-parameter /hrr_cobot/sensitive_grasping/rotation_velocity. Use default 0.05 instead.
[rosout] [/grasping_client] could not get ROS-parameter /hrr_cobot/sensitive_grasping/grasp_timeout. Use default 4.0 instead.
[rosout] Received new goal for sensitive_grasping-skill.


True

In [3]:
tf_msg = hrr_common.get_empty_msg_stamped("hrr_cobot.wsg_50_tcp", "hrr_cobot.wsg_50_dsa_center")

In [4]:
self = grasping
self.B_object_des = T_B_object_des = sm.SE3(0.66, 0.0, 0.05)
T_B_E_goal = hrr_common.calc_EE_goal_pose(B_normal=self.B_surface_normal, B_y_axis=self.B_tcp_y_axis,
                                          T_C_E=self.cobot.T_E_C_robot.inv(),
                                          B_p_location=self.B_object_des.t)
self._T_B_E_goal = T_B_E_goal

In [5]:
self.recovery()

In [23]:
def approach_object():
    self.cobot.init_sns_vel()
    self.cobot.gripper.set_vel(0.0)
    rospy.sleep(0.1)
    self.cobot.gripper.send_pos(self._open_width, si=True)
    self.process_steps((
        SimpleMp(name="approach_pre_pose", f_init=self.cobot.move_to_pose,
                 args=(sm.SE3(self.hover_distance * self.B_surface_normal) @ T_B_E_goal)),
        SimpleMp(name="approach_surface", f_init=self.cobot.set_py_hybrid_force_vel_command,
                 kwargs=approach_kwargs,
                 contact_as_success=True)
    ))

def first_grasp():
    self.update_feedback_msg("Initiating gripper closure")
    self.strategies = self.strategies
    self.cobot.gripper.set_vel(-np.abs(self.closing_speed), si=True)
    if not self.grasp():
        self.failure = True

def open_fingers(dt=0.01):
    self.update_feedback_msg("release object")
    self.cobot.gripper.set_vel(0.5 * np.abs(self.closing_speed), si=True)
    self.cobot.set_compliant(np.r_[1, np.zeros(4), 1])
    while self.cobot.gripper.dsa_sum[0] > 0 or self.cobot.gripper.dsa_sum[1] > 0:
        rospy.sleep(dt)
        self.cobot.update()
    self.cobot.gripper.set_vel(0.0, si=True)
        
def reposition_gripper():
    self.process_steps((
        SimpleMp(name="vertical_reposition_contact_search", f_init=self.cobot.set_py_hybrid_force_vel_command,
                 kwargs=approach_kwargs, contact_as_success=True),
        SimpleMp(name="vertical_reposition", f_init=self.cobot.set_py_hybrid_force_vel_command,
                 kwargs=move_up_kwargs, T_max=1.5),
    ))
    self.cobot.gripper.set_vel(-np.abs(self.closing_speed), si=True)
    self.update_feedback_msg("Initiating gripper closure")
    if not self.grasp():
        self.failure = True
    self.cobot.gripper.set_vel(0.0, si=False)

def rotate_open(steps):
    self.grasp()
    for t in range(steps):
        rotate_vel[4] = 0.05
        tf_msg.transform.translation.z = self.grasping_controller.err_cur[2] if \
            self.grasping_controller.err_cur[2] != 0.0 else tf_msg.transform.translation.z
        self._tf.send_tf(tf_msg)
        self.cobot.update(u_cmd=rotate_vel, u_cmd_frame=tf_msg.child_frame_id)
    self.cobot.stop()
    self.grasp()

def rotate_back(steps):
    for t in range(steps):
        rotate_vel[4] = -0.05
        tf_msg.transform.translation.z = self.grasping_controller.err_cur[2] if \
            self.grasping_controller.err_cur[2] != 0.0 else tf_msg.transform.translation.z
        self._tf.send_tf(tf_msg)
        self.cobot.update(u_cmd=rotate_vel, u_cmd_frame=tf_msg.child_frame_id)
    self.cobot.stop()

In [24]:
def debug_skill():
    self.grasping_controller.strategies[0] = self.grasping_controller.F_CTRL
    self.failure = False
    approach_object()
    if self.failure: 
        print("failed during approach")
        return
    first_grasp()    
    if self.failure: 
        print("failed during initial grasp")
        return
    open_fingers()
    reposition_gripper()

In [25]:
self.failure = False
self.success = False
self.f_contact = 5.0
self._T_B_E_goal = T_B_E_goal
self.cobot.init_sns_vel()
self.cobot.gripper.set_vel(0.0)
rospy.sleep(0.1)
self.cobot.gripper.send_pos(self._open_width, si=True)
approach_kwargs = self.cobot.default_hybrid_kwargs()
approach_kwargs["scale_pos"] = self.scale_pos_vel
move_up_kwargs = copy.deepcopy(approach_kwargs)
approach_kwargs["vel_dir"] = np.r_[-self.B_surface_normal, np.zeros(3)]
move_up_kwargs["vel_dir"] = np.r_[self.B_surface_normal, np.zeros(3)]
tf_msg = hrr_common.get_empty_msg_stamped(self.tcp_link, self.dsa_link)
rotate_vel = np.zeros(6)
cur_angle = 0.0

In [26]:
debug_skill()

KeyboardInterrupt: 

In [22]:
open_fingers()

In [43]:
reposition_gripper()

In [29]:
rotate_open(200)

In [14]:
self.recovery()

In [15]:
self.cobot.B_F_msr

array([ 1.49334, -0.38275,  0.61424,  1.54254, -1.26021, -1.31179])

In [54]:
open_fingers()

In [13]:
self.action_sever_valid, self.cobot.state

(True, 'FAILURE')

In [86]:
self._tf.send_tf()

In [78]:
self.closing_width, self.cobot.gripper.width

(0.02, 0.02320912852883339)

In [47]:
rotate_back(100)

In [27]:
self.recovery()

In [46]:
rotate_back(400)

In [31]:
import quaternion

0.0

In [48]:
cur_angle = 0
q0 = quaternion.from_rotation_matrix(self.cobot.T_B_C_robot.R).copy()
while cur_angle < self._tilting_angle:
    rotate_open(500)
    cur_angle += hrr_common.quaternion_distance(quaternion.from_rotation_matrix(self.cobot.T_B_C_robot.R), q0)
    open_fingers()
    rotate_back(500)

In [42]:
 quaternion.from_rotation_matrix(self.cobot.T_B_C_robot.R),q0


(quaternion(-0.157622949137034, 0.0187112680230015, 0.987320748298061, 0.00162306329916049),
 array(quaternion(0.0160000221904193, 0.0187103222423431, 0.99969557241438, -0.00163878814308873), dtype=quaternion))

In [45]:
self._tilting_angle = np.pi / 2.0